In [1]:
import pandas as pd
import json
import requests
import time
from ApiKey import key1, key2
import datetime



In [2]:
baseurl = 'https://api.usa.gov/crime/fbi/sapi/'
agencyapi = 'api/agencies'

params = {'api_key' : key1}


counties = pd.read_csv('bureau_of_labor_counties_with_data.csv')
counties['cname'] = counties['County Name'].str.replace(' County', '').str.upper()
counties.head()

,Area Type,Area Code,County Name,State,Display Level,Selectable,Sort Sequence,cname
0,F,CN0108100000000,Lee County,AL,0,T,71,LEE
1,F,CN0100700000000,Bibb County,AL,0,T,34,BIBB
2,F,CN0102700000000,Clay County,AL,0,T,44,CLAY
3,F,CN0104500000000,Dale County,AL,0,T,53,DALE
4,F,CN0106500000000,Hale County,AL,0,T,63,HALE


In [3]:
oris = requests.get(baseurl + agencyapi, params=params).json() #Retrieve all agencies from source
# print(json.dumps(oris, indent=2, sort_keys=True))

In [4]:
# crimedata = requests.get(baseurl + '/api/summarized/agencies/AK0010200/offenses', params)
# crimedata
# mydat =

In [5]:
# crimedata.json()

In [6]:
mydata = {'Agency':[], 'Year':[], 'Offense':[],'count':[], 'county':[], 'state':[]}
print(f"starting at {datetime.datetime.now()}")
x = 0
for mystate in oris:
    if x < 25:
        x+=1
        print(mystate)
    #     print(counties[counties['State'] == mystate]['State'].count())
        if counties[counties['State'] == mystate]['State'].count() > 0: #Check to see if state in county list
            statecounties = counties[counties['State'] == mystate]
    #         print(mystate)
        #     print(oris[mystate])
            for myori in oris[mystate]:

    #             print(statecounties[statecounties['cname'] ==  oris[mystate][myori]['county_name']])
                if  statecounties[statecounties['cname'] ==  oris[mystate][myori]['county_name']]['cname'].count() > 0: #Check to see if county in list
                    print('     ' + oris[mystate][myori]['county_name'])
                    crimedata = ''        
                    while crimedata == '':
                        try:
                            oriapi = f'/api/summarized/agencies/{myori}/offenses'
                            crimedata = requests.get(baseurl + oriapi, params).json()
                            crimedata['results']
                            for result in crimedata['results']:
                                if  2010 <= result['data_year']  <= 2015:
                                    mydata['Agency'].append(myori)
                                    mydata['Year'].append(result['data_year'])
                                    mydata['Offense'].append(result['offense'])
                                    mydata['count'].append(result['actual'])
                                    mydata['county'].append(oris[mystate][myori]['county_name'])
                                    mydata['state'].append(mystate)
                        except KeyError:
                            crimedate = ''
                            print('waiting 15 minutes')
                            print(datetime.datetime.now())
                            time.sleep(901)
                            if params['api_key'] == key1:
                                params['api_key'] = key2
                            else:
                                params['api_key'] = key1

    else:
        print('do nothing')

        
            
        
        

starting at 2018-11-04 17:36:55.912708
HI
     HAWAII
     MAUI
     KAUAI
DE
     SUSSEX
     KENT
     SUSSEX
     SUSSEX
     SUSSEX
     KENT
     KENT
     SUSSEX
     NEW CASTLE
     NEW CASTLE
     KENT
     NEW CASTLE
     NEW CASTLE
     NEW CASTLE
     NEW CASTLE
     SUSSEX
     KENT
     SUSSEX
     NEW CASTLE
     KENT
     SUSSEX
     SUSSEX
     KENT
     KENT
     KENT
     SUSSEX
     KENT
     NEW CASTLE
     SUSSEX
     SUSSEX
     SUSSEX
     SUSSEX
     KENT
     KENT
     SUSSEX
     SUSSEX
     NEW CASTLE
     KENT
     NEW CASTLE
     NEW CASTLE
     NEW CASTLE
     NEW CASTLE
     SUSSEX
     SUSSEX
     KENT
     SUSSEX
     KENT
     KENT
     SUSSEX
     KENT
     SUSSEX
     KENT
     KENT
     SUSSEX
     KENT
     NEW CASTLE
     SUSSEX
     NEW CASTLE
     NEW CASTLE
PR
TX
     WASHINGTON
     BRAZORIA
     EDWARDS
     HARRIS
     MILAM
     MARION
     WILSON
     DENTON
     GALVESTON
     HARRIS
     BRAZORIA
     CALHOUN
     COLORADO
     HARRIS
  

     HARDIN
     MADISON
     EL PASO
     MCLENNAN
     VAL VERDE
     BEXAR
     STARR
     ZAVALA
     BRAZORIA
     DENTON
     CHEROKEE
     RANDALL
     UVALDE
     SHACKELFORD
     ROBERTS
     NAVARRO
     HARRIS
     COMANCHE
     STERLING
     CAMERON
     MARION
     BRAZORIA
     DENTON
     WILSON
     ORANGE
     LAVACA
     HENDERSON
     CALHOUN
     MASON
     HARRIS
     HARRIS
     TARRANT
     WILLIAMSON
     EL PASO
     JIM WELLS
     HIDALGO
     BEXAR
     HILL
     EASTLAND
     HASKELL
     COLLIN
     DALLAS
     JOHNSON
     LAMPASAS
     NUECES
     EL PASO
     BELL
     EASTLAND
     COLLIN
     WICHITA
     CRANE
     HUNT
     TARRANT
     TERRELL
     DUVAL
     MEDINA
     ELLIS
     BEXAR
     ANDERSON
     MORRIS
     COLLINGSWORTH
     UPSHUR
     HARDIN
     DEAF SMITH
     BLANCO
     EL PASO
     SHELBY
     BRAZORIA
     MONTGOMERY
     WALLER
     MADISON
     HUNT
     RANDALL
     GUADALUPE
     WOOD
     BEXAR
     REEVES
     LUBBOCK
     

     MIDDLESEX
     PLYMOUTH
     FRANKLIN
     WORCESTER
     FRANKLIN
     HAMPSHIRE
     NORFOLK
     ESSEX
     MIDDLESEX
     NORFOLK
     HAMPDEN
     WORCESTER
     BRISTOL
     WORCESTER
     HAMPDEN
     FRANKLIN
     WORCESTER
     PLYMOUTH
     PLYMOUTH
     SUFFOLK
     DUKES
     MIDDLESEX
     HAMPDEN
     ESSEX
     ESSEX
     SUFFOLK
     HAMPDEN
     NORFOLK
     WORCESTER
     WORCESTER
     WORCESTER
     MIDDLESEX
     MIDDLESEX
     FRANKLIN
     PLYMOUTH
     BERKSHIRE
     MIDDLESEX
     HAMPDEN
     ESSEX
     PLYMOUTH
     NORFOLK
     WORCESTER
     WORCESTER
     BRISTOL
     FRANKLIN
     WORCESTER
     MIDDLESEX
     PLYMOUTH
     SUFFOLK
     DUKES
     ESSEX
     ESSEX
     SUFFOLK
     HAMPDEN
     WORCESTER
     BRISTOL
     WORCESTER
     WORCESTER
     FRANKLIN
     FRANKLIN
     WORCESTER
     PLYMOUTH
     SUFFOLK
     ESSEX
     NORFOLK
     MIDDLESEX
     HAMPDEN
     HAMPDEN
     WORCESTER
     PLYMOUTH
     PLYMOUTH
     WORCESTER
     MIDDLESEX

     ALLAMAKEE
     CLINTON
     JACKSON
     BENTON
     POLK
     HENRY
     SCOTT
     DECATUR
     LINN
     JOHNSON
     SHELBY
     MADISON
     BREMER
     TAMA
     POLK
     GRUNDY
     DALLAS
     JASPER
     STORY
     KOSSUTH
     CEDAR
     BUTLER
     LUCAS
     WORTH
     FAYETTE
     OSCEOLA
     MUSCATINE
     MAHASKA
     DES MOINES
     BLACK HAWK
     KEOKUK
     JONES
     WOODBURY
     BENTON
     LINN
     CARROLL
     SCOTT
     MILLS
     JOHNSON
     CHEROKEE
     IOWA
     DAVIS
     WRIGHT
     CHEROKEE
     MONTGOMERY
     POLK
     DICKINSON
     FLOYD
     PAGE
     ALLAMAKEE
     JASPER
     WAYNE
     HARDIN
     HUMBOLDT
     VAN BUREN
     STORY
     FAYETTE
     MUSCATINE
     HOWARD
     SIOUX
     JONES
     CASS
     BUCHANAN
     LOUISA
     MAHASKA
     UNION
     SCOTT
     MILLS
     LINN
     CERRO GORDO
     WINNESHIEK
     SCOTT
     CLAYTON
     DAVIS
     SAC
     MONTGOMERY
     DICKINSON
     CERRO GORDO
     POLK
     HARRISON
     TAM

     LENAWEE
     LIVINGSTON
     BAY
     LAPEER
     ARENAC
     ALLEGAN
     MUSKEGON
     OAKLAND
     BARAGA
     ALLEGAN
     GENESEE
     GENESEE
     MASON
     MIDLAND
     GENESEE
     KALAMAZOO
     MONROE
     CHARLEVOIX
     SAGINAW
     EATON
     CHARLEVOIX
     VAN BUREN
     MACOMB
     MONTCALM
     BRANCH
     CRAWFORD
     MACOMB
     INGHAM
     BARAGA
     ARENAC
     MONTCALM
     TUSCOLA
     LENAWEE
     DELTA
     MARQUETTE
     WEXFORD
     OCEANA
     OAKLAND
     CRAWFORD
     BRANCH
     MASON
     CHEBOYGAN
     WEXFORD
     IRON
     CALHOUN
     EATON
     WAYNE
     LEELANAU
     OAKLAND
     MECOSTA
     KEWEENAW
     VAN BUREN
     MUSKEGON
     MANISTEE
     MACOMB
     GENESEE
     BERRIEN
     ANTRIM
     CASS
     ALGER
     OCEANA
     BARRY
     SHIAWASSEE
     WAYNE
     WASHTENAW
     IOSCO
     OAKLAND
     MECOSTA
     SHIAWASSEE
     LUCE
     KALAMAZOO
     SAGINAW
     BERRIEN
     BARRY
     IOSCO
     CHARLEVOIX
     IRON
     GENESEE


...

     KANE
     WHITESIDE
     SANGAMON
     COOK
     LAKE
     COOK
     MADISON
     CLINTON
     JASPER
     ADAMS
     DUPAGE
     SANGAMON
     OGLE
     COOK
     LAKE
     COOK
     CLARK
     WINNEBAGO
     PEORIA
     COOK
     CHAMPAIGN
     COLES
     SANGAMON
     CHRISTIAN
     TAZEWELL
     PUTNAM
     JO DAVIESS
     MCLEAN
     LAKE
     COOK
     KANE
     ROCK ISLAND
     UNION
     WILL
     SALINE
     KNOX
     MCHENRY
     FAYETTE
     WILLIAMSON
     TAZEWELL
     COOK
     MACOUPIN
     MCLEAN
     COOK
     KANE
     FULTON
     MONTGOMERY
     MARSHALL
     WINNEBAGO
     WILL
     MACON
     BUREAU
     WHITESIDE
     MASON
     CALHOUN
     MCHENRY
     DE WITT
     FRANKLIN
     DEKALB
     COOK
     COOK
     SANGAMON
     LAKE
     CLINTON
     COOK
     MOULTRIE
     MARION
     MCHENRY
     DUPAGE
     SANGAMON
     COOK
     LAKE
     MCDONOUGH
     WILL
     OGLE
     CHAMPAIGN
     COOK
     CHRISTIAN
     DUPAGE
     VERMILION
     MCLEAN
     COOK


     LAKE
     COOK
     COOK
     KANE
     COOK
     HENRY
     MADISON
     DUPAGE
     COOK
     SHELBY
     ADAMS
     LAKE
     EDGAR
     DOUGLAS
     COOK
     LASALLE
     JACKSON
     COOK
     VERMILION
     LAKE
     PEORIA
     COOK
     RICHLAND
     LASALLE
     COOK
     WASHINGTON
     COOK
     CHAMPAIGN
     MADISON
     LAWRENCE
     LAKE
     WAYNE
     BUREAU
     GREENE
     COOK
     LIVINGSTON
     WINNEBAGO
     LEE
     KANKAKEE
     BOND
     COOK
     ROCK ISLAND
     DUPAGE
     MACOUPIN
     EFFINGHAM
     WILL
     COOK
     CHAMPAIGN
     MADISON
     LAKE
     STARK
     KANE
     MONTGOMERY
     MONROE
     MACON
     COLES
     STEPHENSON
     DUPAGE
     ROCK ISLAND
     DEKALB
     VERMILION
     COOK
     LASALLE
     JO DAVIESS
     JACKSON
     VERMILION
     COOK
     COOK
     KANE
     WINNEBAGO
     LASALLE
IN
     DELAWARE
     SHELBY
     GIBSON
     HANCOCK
     LAKE
     RUSH
     RIPLEY
     HOWARD
     SULLIVAN
     PERRY
     PORTER
 

     WARREN
     LAFAYETTE
     PIKE
MT
     RAVALLI
     GALLATIN
     LAKE
     CHOUTEAU
     PONDERA
     MISSOULA
     TETON
     FLATHEAD
     RICHLAND
     PARK
     LEWIS AND CLARK
     MADISON
     GALLATIN
     JEFFERSON
     TREASURE
     POWELL
     CARTER
     PHILLIPS
     CUSTER
     SWEET GRASS
     FERGUS
     LINCOLN
     GLACIER
     CARBON
     ROSEBUD
     PRAIRIE
     GALLATIN
     LAKE
     SHERIDAN
     MISSOULA
     SANDERS
     RAVALLI
     FLATHEAD
     GRANITE
     POWDER RIVER
     CASCADE
     WIBAUX
     DEER LODGE
     BLAINE
     MADISON
     GALLATIN
     CARBON
     STILLWATER
     JEFFERSON
     ROOSEVELT
     YELLOWSTONE
     MINERAL
     FERGUS
     ROSEBUD
     LINCOLN
     BEAVERHEAD
waiting 15 minutes
2018-11-04 20:12:07.728757
     GALLATIN
     DAWSON
     GALLATIN
     LAKE
     MUSSELSHELL
     SANDERS
     RAVALLI
     FLATHEAD
     GARFIELD
     HILL
     TOOLE
     CASCADE
     LEWIS AND CLARK
     BLAINE
     RICHLAND
     PETROLEUM
     

     ETOWAH
     COVINGTON
     COFFEE
     PERRY
     BUTLER
     JEFFERSON
     HOUSTON
     BLOUNT
     LAUDERDALE
     MOBILE
     CALHOUN
     MADISON
     MARSHALL
     LAWRENCE
     GENEVA
     FRANKLIN
     WALKER
     MONTGOMERY
     MARENGO
     CLEBURNE
     WASHINGTON
     MOBILE
     RANDOLPH
     DALE
     MARION
     COVINGTON
     RUSSELL
     JEFFERSON
     JEFFERSON
     MOBILE
     CALHOUN
     LAUDERDALE
     MARSHALL
     LAWRENCE
     CULLMAN
     CLAY
     MADISON
     GENEVA
     MACON
     CLEBURNE
     BULLOCK
     GREENE
     DALE
     MOBILE
     RANDOLPH
     WASHINGTON
     COVINGTON
     JACKSON
     BUTLER
     JEFFERSON
     JEFFERSON
     MOBILE
     TALLAPOOSA
     BLOUNT
     CALHOUN
     LAUDERDALE
     LAWRENCE
     MADISON
     CLAY
     CULLMAN
     FRANKLIN
     WALKER
     MONROE
     CLARKE
     MACON
     CHAMBERS
     JEFFERSON
     MARENGO
     DEKALB
     WASHINGTON
     MOBILE
     DALE
     BARBOUR
     ETOWAH
     BALDWIN
     DALLAS
  

     MITCHELL
     CUMBERLAND
     HAYWOOD
     CATAWBA
     WAYNE
     MECKLENBURG
     ASHE
     PITT
     WATAUGA
     PERSON
     NORTHAMPTON
     HARNETT
     DAVIE
     WAKE
     BEAUFORT
     ONSLOW
     RUTHERFORD
     CRAVEN
     COLUMBUS
     PERQUIMANS
     MOORE
     CARTERET
     HALIFAX
     ROWAN
     HENDERSON
     ALEXANDER
     LEE
     WAKE
     TRANSYLVANIA
     BRUNSWICK
     NEW HANOVER
     ALAMANCE
     ROBESON
     HAYWOOD
     CATAWBA
     WAYNE
     JONES
     AVERY
     HARNETT
     CUMBERLAND
     PAMLICO
     SURRY
     BRUNSWICK
     WASHINGTON
     PERQUIMANS
     CUMBERLAND
     JOHNSTON
     RUTHERFORD
     HALIFAX
     COLUMBUS
     ORANGE
     MOORE
     CARTERET
     STANLY
     CRAVEN
     ONSLOW
     ROWAN
     HENDERSON
     ALEXANDER
     WAKE
     LEE
     TRANSYLVANIA
     BRUNSWICK
     ROBESON
     NASH
     BLADEN
     NEW HANOVER
     ALAMANCE
     WAYNE
     HAYWOOD
     WATAUGA
     HAYWOOD
     ALAMANCE
     GASTON
     DAVIE
     BRUNS

In [7]:
# crimedata['results']
# 

In [8]:
# agencies = pd.read_csv('./data/agencies.csv')
# # agencies.head()
# print(agencies.loc[agencies['agency_id']==61][['ncic_agency_name','agency_type_name', 'population']])
# agencies.columns
# agencies_rev=agencies[['agency_id','county_name']]
# agencies_rev

In [9]:
# incidents = pd.read_csv('./data/NIBRS_incident.csv')
# incidents.head()
# incidents.dtypes
# incidents_rev = incidents[['agency_id','incident_date']]
# incidents_rev.dropna(axis=0, how='any',inplace=True)
# incidents_rev['AGENCY_ID'] =incidents_rev['agency_id'].astype('int64')
# incidents_rev.dtypes

In [10]:
# merge_data = incidents_rev.merge(agencies_rev, left_on='AGENCY_ID', right_on='agency_id', how='inner')
# merge_data

In [13]:
cdata = pd.DataFrame(mydata)
cdata.head()
crimegb = cdata.groupby(['Year','county','state']).sum()
crimegb.reset_index(inplace=True)
crimegb


,Year,county,state,count
0,2010,ACCOMACK,VA,1348
1,2010,ADA,ID,18217
2,2010,ADAIR,IA,120
3,2010,ADAIR,MO,1684
4,2010,ADAMS,IA,168
5,2010,ADAMS,ID,122
6,2010,ADAMS,IL,3559
7,2010,ADAMS,IN,218
8,2010,ADAMS,MS,3393
9,2010,ADAMS,ND,24


In [14]:
crimegb.to_csv('Crimedata.csv')